In [2]:
%load_ext autoreload
%autoreload 2
import torch
from torch import nn


/system/apps/userenv/beck/rwkv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using /system/user/beck/.cache/torch_extensions/py310_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /system/user/beck/.cache/torch_extensions/py310_cu117/wkv/build.ninja...
Building extension module wkv...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module wkv...


## Understand single operations inside RWKV model

### Time shift operation

In [ ]:
zp = nn.ZeroPad2d((0, 0, 1, -1))

In [ ]:
t = torch.randn(2, 10, 5)
t

tensor([[[ 2.2994,  0.6164, -0.2690,  0.4272,  0.0744],
         [-0.9279, -1.5175,  1.2302,  1.2907, -1.4773],
         [ 0.1119,  2.7972,  0.2340, -0.5418,  0.3708],
         [ 0.8383, -0.6947,  0.1652, -1.0719, -1.8091],
         [ 0.1493, -0.9867,  0.2290,  1.0927, -0.1701],
         [ 0.7025, -0.6657, -1.0994, -0.7444,  0.5355],
         [-1.8331, -0.4958,  0.3811, -0.1416,  0.0564],
         [-1.6845,  1.4969,  2.0742, -0.6748,  2.6469],
         [ 1.3789,  0.0776,  1.1843,  0.2751,  1.3776],
         [-1.4278, -0.5132, -0.4568, -0.7256, -0.4097]],

        [[ 0.6982, -0.5857, -0.3218, -2.6634,  0.6578],
         [ 0.5408, -0.4575,  0.2834,  1.1029,  0.1854],
         [-0.6689,  1.0695, -2.1253, -0.5972,  0.2518],
         [-1.8999, -0.2150, -1.3322,  1.5590, -0.9475],
         [ 0.1050, -0.3192,  0.0122, -0.4665, -0.7289],
         [ 1.3017, -1.0393, -1.3678, -0.2464, -0.7555],
         [-1.3445,  1.5817, -0.5817,  0.5974, -0.0958],
         [ 0.0687,  0.1729, -0.6784,  0.4652, 

In [ ]:
zp(t).shape

torch.Size([2, 10, 5])

In [ ]:
zp(t)

tensor([[[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 2.2994,  0.6164, -0.2690,  0.4272,  0.0744],
         [-0.9279, -1.5175,  1.2302,  1.2907, -1.4773],
         [ 0.1119,  2.7972,  0.2340, -0.5418,  0.3708],
         [ 0.8383, -0.6947,  0.1652, -1.0719, -1.8091],
         [ 0.1493, -0.9867,  0.2290,  1.0927, -0.1701],
         [ 0.7025, -0.6657, -1.0994, -0.7444,  0.5355],
         [-1.8331, -0.4958,  0.3811, -0.1416,  0.0564],
         [-1.6845,  1.4969,  2.0742, -0.6748,  2.6469],
         [ 1.3789,  0.0776,  1.1843,  0.2751,  1.3776]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.6982, -0.5857, -0.3218, -2.6634,  0.6578],
         [ 0.5408, -0.4575,  0.2834,  1.1029,  0.1854],
         [-0.6689,  1.0695, -2.1253, -0.5972,  0.2518],
         [-1.8999, -0.2150, -1.3322,  1.5590, -0.9475],
         [ 0.1050, -0.3192,  0.0122, -0.4665, -0.7289],
         [ 1.3017, -1.0393, -1.3678, -0.2464, -0.7555],
         [-1.3445,  1.5817, -0.5817,  0.5974, 

In [ ]:
zp(t)[0, 1, :]

tensor([ 2.2994,  0.6164, -0.2690,  0.4272,  0.0744])

In [ ]:
t[0, 0, :]

tensor([ 2.2994,  0.6164, -0.2690,  0.4272,  0.0744])

### Init of time_first 

In [3]:
attention_dim = 10
zigzag = torch.tensor([(i + 1) % 3 - 1
                       for i in range(attention_dim)]) * 0.5

In [4]:
zigzag

tensor([ 0.0000,  0.5000, -0.5000,  0.0000,  0.5000, -0.5000,  0.0000,  0.5000,
        -0.5000,  0.0000])